In [1]:
import pandas as pd
import numpy as np
import requests
import json

# Notebook de contôle

## Contrôles à appliquer avant le workflow Dagster

In [2]:
observation_date = "2022-08-29"
corpus_end_year = 2022

In [18]:
df = pd.read_json(f'01_raw/{observation_date}/exportDonnees_barometre_complet_{observation_date}.json')
df.shape

(43309, 20)

### Vérif fichier bien formé

In [4]:
df[df["dc:identifiers"] == "SCOPUS_ID:84961134992"][["dc:identifiers","@auid","@afids","ce:indexed-name"]]

,dc:identifiers,@auid,@afids,ce:indexed-name
10805,SCOPUS_ID:84961134992,56656653900,60110764,Rageot M.
10806,SCOPUS_ID:84961134992,56656653900,60110763,Rageot M.
10807,SCOPUS_ID:84961134992,6603126272,60110764,Regert M.
10808,SCOPUS_ID:84961134992,7003877322,60110763,Filippi J.J.
10809,SCOPUS_ID:84961134992,7004449219,60110763,Fernandez X.


### Vérif affiliations à ajouter

In [19]:
data = df[["dc:identifiers","prism:doi","reference","annee_pub","@afids","mentionAffil_reconstruct","@auid","ce:indexed-name","corresponding_author","Is_dc:creator"]]
data.columns = ['source_id', 'doi',"scopus_title",'year', 'aff_scopus_id','aff_source_text','author_id','author_name','corresponding_author','creator_author']
df_authors = data.groupby('source_id')['author_name'].apply(list).reset_index(name='all_authors')
df_authors['all_authors'] = df_authors["all_authors"].apply('|'.join)
df_reference_data = pd.merge(data,df_authors, left_on='source_id', right_on='source_id')

In [18]:
df_affiliations = pd.read_json('03_primary/referentiel_structures.json')
set_affiliations_id = set(df_affiliations['affiliation_id'].tolist())
set_data_affiliations_id = set(df_reference_data['aff_scopus_id'].unique().tolist())
diff = set_affiliations_id.union(set_data_affiliations_id)  - set_affiliations_id.intersection(set_data_affiliations_id) 
diff

{'0', 'temp_008', 'temp_009'}

### Si diff != 1

In [20]:
#exemple pour retrouver le libellé d'un aff_scopus_id
df[df["@afids"] == "temp_009"].mentionAffil_reconstruct

18258    MajuLab, CNRS, Université de Nice, NUS-NTU Int...
19714    MajuLab., CNRS-Université Nice Sophia Antipoli...
23787    MajuLab, CNRS, Université de Nice, NUS-NTU Int...
23788    MajuLab, CNRS, Université de Nice, NUS-NTU Int...
23885    MajuLab, CNRS-UNS-NUS-NTU, Université Côte d'A...
                               ...                        
42604    MajuLab, International Joint Research Unit IRL...
42951    MajuLab, International Joint Research Unit IRL...
42952    MajuLab, International Joint Research Unit IRL...
42991    MajuLab, International Joint Research Unit UMI...
43302    CRHEA-CNRS, Rue Bernard Gregory| MajuLab, Inte...
Name: mentionAffil_reconstruct, Length: 104, dtype: object

- Ajouter à la main les nouvelles structures 03_primary/referentiel_structures.json
- Compléter avec les ids trouvés (Hal, Idref...)
- Incrémenter le champ id
- Vérifier le parent_id
- Mettre le nb documents_count à 0

### Vérifs éditeurs à ajouter

In [27]:
def keep_duplicate (row):
    return row["author_name"] +"|" + row["affiliation_name"]
crfprefix_base_url = "https://api.crossref.org/v1/prefixes/"
def crf_publisher_metadata(prefix):
    """Get the homogeneous publisher's name from a prefix doi"""
    if prefix is None:
        raise ValueError('prefix cannot be None')
    result = {}
    result["prefix"] = prefix
    try:
        requests.get(crfprefix_base_url+str(prefix))
        if requests.get(crfprefix_base_url+str(prefix)).status_code == 200:
            response = requests.get(crfprefix_base_url+str(prefix)).text
            result["publisher_by_doiprefix"] = json.loads(response).get("message")["name"]
        else:
            pass
    except:
        pass
    return result

def crf_publisher_retrieval(doiprefix_list):
    """Request function crf_publisher_metadata from a list of doi prefixs and compile in a dataframe"""
    df_result = pd.DataFrame(crf_publisher_metadata(i) for i in doiprefix_list)
    return df_result[df_result["prefix"].notna()]

In [21]:
df_affiliations = pd.read_json('03_primary/referentiel_structures.json')

In [22]:
df_affiliations["affiliation_id"] = df_affiliations["affiliation_id"].astype('str')
df_reference_data["aff_scopus_id"] = df_reference_data["aff_scopus_id"].astype('str')
publis_all_with_affiliations_data = pd.merge(df_reference_data,df_affiliations[df_affiliations["affiliation_id"].notna()], left_on='aff_scopus_id', right_on='affiliation_id',how="left").drop(columns=['affiliation_id','documents_count','ppn_valide','affcourt_valide','RNSR','VIAF','ISNI','BNF','HAL'])
publis_all_with_affiliations_data = publis_all_with_affiliations_data.rename(columns={'id': 'aff_internal_id', 'parent_id': 'aff_parent_id'})
# identify corresponding author if UCA
publis_all_with_affiliations_data["corresponding"] = publis_all_with_affiliations_data[publis_all_with_affiliations_data["corresponding_author"] == "oui"].apply (lambda row: keep_duplicate(row), axis=1)

In [23]:
publis_all_with_affiliations_data["corresponding_author"] = publis_all_with_affiliations_data["corresponding_author"].astype('category')
publis_all_with_affiliations_data["corresponding_author"] = publis_all_with_affiliations_data["corresponding_author"].cat.set_categories(['oui', 'non', 'corr absent pour cette publi'], ordered=True)
publis_all_with_affiliations_data.sort_values(by=['doi', 'corresponding_author'])
publis_uniques_doi_data = publis_all_with_affiliations_data[publis_all_with_affiliations_data.doi.notna()].drop_duplicates(subset=['doi'], keep='first')[["source_id","doi","year","corresponding","all_authors"]]
publis_uniques_doi_data = publis_uniques_doi_data[publis_uniques_doi_data.year < int(corpus_end_year)]

In [24]:
new_prefix_list = list(set([item.partition("/")[0] for item in publis_uniques_doi_data["doi"].to_list()]))
old_prefix_df = pd.read_csv('03_primary/mapping_doiprefixes_publisher.csv', sep=",",encoding='utf8')
old_prefix_list = old_prefix_df["prefix"].astype(str).to_list()
diff_prefix_list = list(set(new_prefix_list) - set(old_prefix_list))
diff_prefix_list

['10.5220',
 '10.7150',
 '10.22564',
 '10.17396',
 '10.1210',
 '10.1200',
 '10.21630',
 '10.3390',
 '10.1130',
 '10.5210',
 '10.25537',
 '10.25428',
 '10.9876',
 '10.14601',
 '10.24820',
 '10.30549',
 '10.22201',
 '10.5441',
 '10.3750',
 '10.1070',
 '10.2383',
 '10.2110',
 '10.5840',
 '10.6092',
 '10.1590',
 '10.5277',
 '10.14658',
 '10.1080',
 '10.21409',
 '10.13128',
 '10.2340',
 '10.2423',
 '10.1190',
 '10.3150',
 '10.1439',
 '10.24310',
 '10.1680',
 '10.19272',
 '10.11936',
 '10.5278',
 '10.20960',
 '10.35470',
 '10.14649',
 '10.57',
 '10.1160',
 '10.11909',
 '10.2312',
 '10.26331',
 '10.4310',
 '10.4418',
 '10.1482',
 '10.26028',
 '10.31009',
 '10.7410',
 '10.48611',
 '10.34190',
 '10.1530',
 '10.1140',
 '10.13130',
 '10.4230',
 '10.3240',
 '10.32473',
 '10.3290',
 '10.21411',
 '10.17660',
 '10.4454',
 '10.1090',
 '10.24840',
 '10.3850',
 '10.4000',
 '10.2140']

In [28]:
df_new_prefix_result = crf_publisher_retrieval(diff_prefix_list)
publishers_doi_prefix = old_prefix_df.append(df_new_prefix_result)

In [29]:
publishers_doi_prefix

,prefix,publisher_by_doiprefix
0,10.21873,Anticancer Research USA Inc.
1,10.1119,American Association of Physics Teachers (AAPT)
2,10.2166,IWA Publishing
3,10.1177,SAGE Publications
4,10.4081,PAGEPress Publications
...,...,...
66,10.1090,American Mathematical Society (AMS)
67,10.24840,University of Porto
68,10.3850,Research Publishing Services
69,10.4000,NaN


In [30]:
publishers_doi_prefix.drop_duplicates(subset=['prefix'], keep='last').to_csv('03_primary/mapping_doiprefixes_publisher.csv', index = False,encoding='utf8')

## Divers

In [3]:
import pickle
import joblib
import helpers.bso_classification_harvest as bso

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\geoffroy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\geoffroy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\geoffroy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [27]:
import importlib
importlib.reload(bso)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\geoffroy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\geoffroy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\geoffroy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


<module 'helpers.bso_classification_harvest' from 'c:\\Users\\geoffroy\\Docker\\dagster\\app\\bso_publis_scopus\\helpers\\bso_classification_harvest.py'>

In [4]:
mesri_bso_dataset = pd.read_csv('https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump/bso-publications-latest.csv.gz',compression='gzip',encoding='utf8').convert_dtypes()

C:\Users\geoffroy\AppData\Local\Temp\ipykernel_8484\2167318211.py:1: DtypeWarning: Columns (0,3,4,5,6,11,12,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  mesri_bso_dataset = pd.read_csv('https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump/bso-publications-latest.csv.gz',compression='gzip',encoding='utf8').convert_dtypes()


In [5]:
publis_uniques_doi_oa_data = pd.read_csv(f'03_primary/{observation_date}/publis_uniques_doi_oa_data.csv', sep=",", encoding="utf-8")

In [6]:
countvect_model = pickle.load(open('06_models/count_vect.pkl', 'rb'))

c:\Users\geoffroy\Docker\dagster\venv_dagster\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
logmodel = joblib.load('06_models/logmodel.joblib')

c:\Users\geoffroy\Docker\dagster\venv_dagster\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
temp = publis_uniques_doi_oa_data[["doi","year","title","journal_name","publisher"]]
#2ème étape : on ajoute une col 
temp["bso_classification"] = np.nan
#3ème étape : on récupère la bso_classification du mesri pour les doi présents
col = 'doi'
cols_to_replace = ['bso_classification']
temp.loc[temp[col].isin(mesri_bso_dataset[col]), cols_to_replace] = mesri_bso_dataset.loc[mesri_bso_dataset[col].isin(temp[col]),cols_to_replace].values

C:\Users\geoffroy\AppData\Local\Temp\ipykernel_8484\2861890832.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["bso_classification"] = np.nan
C:\Users\geoffroy\AppData\Local\Temp\ipykernel_8484\2861890832.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[temp[col].isin(mesri_bso_dataset[col]), cols_to_replace] = mesri_bso_dataset.loc[mesri_bso_dataset[col].isin(temp[col]),cols_to_replace].values


In [9]:
temp1 = temp[(temp.bso_classification.isna()) & (temp.title.notna()) & (temp.publisher.notna()) & (temp.journal_name.notna())]

In [16]:
temp1.shape

(3467, 22)

In [10]:
temp2 = bso.nlp_transform(temp1)

c:\Users\geoffroy\Docker\dagster\app\bso_publis_scopus\helpers\bso_classification_harvest.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  df[i] = df[i].str.replace('\d+', '')
c:\Users\geoffroy\Docker\dagster\app\bso_publis_scopus\helpers\bso_classification_harvest.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i].str.replace('\d+', '')
c:\Users\geoffroy\Docker\dagster\app\bso_publis_scopus\helpers\bso_classification_harvest.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [11]:
temp2["features_union"] = temp2["title_cleaned"] + ' '  +  temp2["journal_name_cleaned"] + ' '  +  temp2["publisher_cleaned"] + ' '  +  temp2["year"].astype(str)

C:\Users\geoffroy\AppData\Local\Temp\ipykernel_8484\3154202690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2["features_union"] = temp2["title_cleaned"] + ' '  +  temp2["journal_name_cleaned"] + ' '  +  temp2["publisher_cleaned"] + ' '  +  temp2["year"].astype(str)


In [12]:
bso_classes = {0: 'Biology (fond.)', 1: 'Chemistry', 2: 'Computer and information sciences', 3: 'Earth, Ecology, Energy and applied biology', 4: 'Engineering',5: 'Humanities', 6: 'Mathematics', 7: 'Medical research', 8: 'Physical sciences, Astronomy', 9: 'Social sciences'} 

In [13]:
temp2["bso_classification"] = temp2.apply(lambda row: bso.to_bso_class_with_ml(row["features_union"],bso_classes,countvect_model,logmodel),axis=1)

C:\Users\geoffroy\AppData\Local\Temp\ipykernel_8484\3128412806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2["bso_classification"] = temp2.apply(lambda row: bso.to_bso_class_with_ml(row["features_union"],bso_classes,countvect_model,logmodel),axis=1)


In [36]:
temp2["bso_classification"]

2                                  Medical research
9                                  Medical research
33                     Physical sciences, Astronomy
36                     Physical sciences, Astronomy
45                     Physical sciences, Astronomy
                            ...                    
14562                                   Engineering
14580    Earth, Ecology, Energy and applied biology
14590                                   Engineering
14601             Computer and information sciences
14602                  Physical sciences, Astronomy
Name: bso_classification, Length: 3467, dtype: object

In [14]:
temp.loc[temp[col].isin(temp2['doi']), ['bso_classification']] = temp2.loc[temp2['doi'].isin(temp['doi']),['bso_classification']].values

In [15]:
temp.bso_classification.value_counts()

Medical research                                3486
Physical sciences, Astronomy                    2498
Biology (fond.)                                 2318
Earth, Ecology, \nEnergy and applied biology    1566
Computer and information sciences                923
Mathematics                                      821
Computer and \n information sciences             804
Social sciences                                  535
Humanities                                       459
Earth, Ecology, Energy and applied biology       418
Chemistry                                        410
Engineering                                      322
Name: bso_classification, dtype: int64

In [17]:
temp.shape

(14607, 6)

In [24]:
test_df = pd.read_csv("07_model_output/2022-08-29/publis_uniques_doi_oa_data_with_bsoclasses.csv",sep=",",encoding="utf-8")
test_df.bso_classification_fr.value_counts()

Recherche médicale                                               3486
Sciences physiques et astronomie                                 2498
Biologie fondamentale                                            2318
Sciences de la Terre, écologie, énergie et biologie appliquée    1984
Informatique et sciences de l'information                        1727
Mathématiques                                                     821
Sciences sociales                                                 535
Sciences humaines                                                 459
Chimie                                                            410
Ingénierie                                                        322
Name: bso_classification_fr, dtype: int64

In [20]:
test_bso_classes = dict({0: 'Biology (fond.)', 1: 'Chemistry', 2: 'Computer and information sciences', 3: 'Earth, Ecology, Energy and applied biology', 4: 'Engineering',5: 'Humanities', 6: 'Mathematics', 7: 'Medical research', 8: 'Physical sciences, Astronomy', 9: 'Social sciences'})

In [21]:
test_bso_classes[7]

'Medical research'